In [5]:
# Library
import sub_class as sc

import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from collections import defaultdict
from datetime import datetime, timezone, timedelta
import os
import gc
import re
import h5py
import exchange_calendars as xcals
import random
import importlib

import torch
import torch.nn as nn
from torch.utils.data import Dataset, get_worker_info, DataLoader

mi = pd.IndexSlice
ST, ED = datetime(2015,1,1), datetime(2024,12,15)
PATH = "e:/Financial_Data/"
H5PATH = PATH + 'data.h5'
SCHEDULE = xcals.get_calendar('XNAS').schedule.loc[ST:ED][['open', 'close']].map(lambda x: x.tz_localize(None) - timedelta(hours=5))
THZ = ['1m', '5m', '30m', '1d', '1w']
CHART_COL_NAME = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']

In [36]:
importlib.reload(sc)

<module 'sub_class' from 'c:\\Users\\naniri\\Documents\\GitHub\\FinanceProject\\sub_class.py'>

In [3]:
def rate(future, past):
  return (future-past) / past * 100

In [ ]:
class SampleModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(SampleModel, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)

In [ ]:
def custom_loss(predictions, targets):
    mse_loss = nn.MSELoss()(predictions, targets)
    abs_error = torch.mean(torch.abs(predictions - targets))
    return mse_loss + 0.1 * abs_error

In [37]:
dataset = sc.StockDatasetHDF5(
    min_peroid=timedelta(weeks=3), min_peroid_label=timedelta(days=10),
    ticker_list=['AAPL', 'MSFT']
)

In [38]:
data = dataset[1]

In [39]:
data['1d']

ohlcv,open,high,low,close,volume,vwap,transactions
timestamp,,,,,,,
2015-01-23,47.360,47.3900,46.8000,47.18,26211594.0,47.1166,109953.0
2015-01-26,47.000,47.1339,46.2399,47.01,42525530.0,46.6059,165070.0
2015-01-27,42.950,43.2000,42.1050,42.66,169162617.0,42.6715,627835.0
2015-01-28,42.740,42.7900,41.1600,41.19,84507078.0,41.9776,317450.0
2015-01-29,40.930,42.1200,40.7900,42.01,63580586.0,41.4486,250395.0
...,...,...,...,...,...,...,...
2024-11-26,419.585,429.0414,418.8500,427.99,22825574.0,425.8382,347024.0
2024-11-27,425.105,427.2299,422.0200,422.99,17407455.0,423.9513,260938.0
2024-11-29,420.090,424.8849,417.8000,423.46,15132529.0,422.3526,210494.0


In [40]:
data['ticker']

'MSFT'